# Grab data drom FEC website. The first section using data mining to grab the previous senate results from the full 3 cycles spanning between 2016-2020. This includes the incumbent status and fec_ids which will be used in step 2.

In [1]:
import requests
from bs4 import BeautifulSoup
import os
import pandas as pd
# interactive pandas tables
%load_ext google.colab.data_table
import zipfile

# download raw excels to a folder - input: year
### https://www.fec.gov/introduction-campaign-finance/election-results-and-voting-information/

In [2]:
def download_excel_to_colab(year, response, directory):

    soup = BeautifulSoup(response.text, 'html.parser')
    links = soup.find_all('a', href=True)

    url2 = 'https://www.fec.gov'
    excel_links = []

    for link in links:
        href = link.get('href')
        if href.endswith(('.xlsx', '.xls')):
            file_url = href if href.startswith('http') else url2 + href
            excel_links.append(file_url)
    link = excel_links[0]

    os.makedirs(directory, exist_ok=True)
    local_filename = os.path.join(directory, f'{year}_raw_senate_results')

    with requests.get(link, stream=True) as r:
        r.raise_for_status()
        with open(local_filename, 'wb') as f:
            for chunk in r.iter_content(chunk_size=8192):
                if chunk:
                    f.write(chunk)

    return print(f'{local_filename} has been successfully saved!!')

In [3]:
years = list(range(2004,2048))[::2]
download_directory = "/raw"
for year in years:
  source_url = f"https://www.fec.gov/introduction-campaign-finance/election-results-and-voting-information/federal-elections-{year}"
  response = requests.get(source_url)
  if response.status_code == 200:
      download_excel_to_colab(year, response, download_directory)
  else:
      print(f"They havn't released info for {year} yet!")

/raw/2004_raw_senate_results has been successfully saved!!
/raw/2006_raw_senate_results has been successfully saved!!
/raw/2008_raw_senate_results has been successfully saved!!
/raw/2010_raw_senate_results has been successfully saved!!
/raw/2012_raw_senate_results has been successfully saved!!
/raw/2014_raw_senate_results has been successfully saved!!
/raw/2016_raw_senate_results has been successfully saved!!
/raw/2018_raw_senate_results has been successfully saved!!
/raw/2020_raw_senate_results has been successfully saved!!
They havn't released info for 2022 yet!
They havn't released info for 2024 yet!
They havn't released info for 2026 yet!
They havn't released info for 2028 yet!
They havn't released info for 2030 yet!
They havn't released info for 2032 yet!
They havn't released info for 2034 yet!
They havn't released info for 2036 yet!
They havn't released info for 2038 yet!
They havn't released info for 2040 yet!
They havn't released info for 2042 yet!
They havn't released info for

# create clean pandas df for each year - input: excel
## (INCUMBENT)

In [4]:
def process_senate_results(year):
  excelfile = f'{year}_raw_senate_results'
  xls = pd.ExcelFile(excelfile)

  keyword = 'Senate Results by State'
  sheet_names = xls.sheet_names
  sheets = []
  for i, sheet_name in enumerate(sheet_names):
      #print(f"{i + 1}. {sheet_name}")
      if keyword in sheet_name:
        sheets.append(sheet_name)

  df = pd.read_excel(excelfile, sheet_name=sheets[0])

  if year == 2016:
    df = df.loc[:, ['STATE ABBREVIATION', 'D', 'FEC ID#', '(I)', 'CANDIDATE NAME (First)', 'CANDIDATE NAME (Last)', 'PARTY', 'GENERAL VOTES ', 'GENERAL %']]
    df = df.rename(columns={
      'STATE ABBREVIATION': 'state',
      'D': 'district',
      'FEC ID#': 'fec_id',
      '(I)': 'incumbent',
      'CANDIDATE NAME (First)': 'first_name',
      'CANDIDATE NAME (Last)': 'last_name',
      'PARTY': 'party',
      'GENERAL VOTES ': 'votes',
      'GENERAL %': 'vote_share'
  })
  elif year == 2018:
    df = df.loc[:, ['STATE ABBREVIATION', 'DISTRICT', 'FEC ID#', '(I)', 'CANDIDATE NAME (First)', 'CANDIDATE NAME (Last)', 'PARTY', 'GENERAL VOTES ', 'GENERAL %']]
    df = df.rename(columns={
      'STATE ABBREVIATION': 'state',
      'DISTRICT': 'district',
      'FEC ID#': 'fec_id',
      '(I)': 'incumbent',
      'CANDIDATE NAME (First)': 'first_name',
      'CANDIDATE NAME (Last)': 'last_name',
      'PARTY': 'party',
      'GENERAL VOTES ': 'votes',
      'GENERAL %': 'vote_share'
  })
  else:
    df = df.loc[:, ['STATE ABBREVIATION', 'DISTRICT', 'FEC ID', '(I) Incumbent Indicator', 'CANDIDATE NAME (First)', 'CANDIDATE NAME (Last)', 'PARTY', 'GENERAL VOTES ', 'GENERAL %']]
    df = df.rename(columns={
      'STATE ABBREVIATION': 'state',
      'DISTRICT': 'district',
      'FEC ID': 'fec_id',
      '(I) Incumbent Indicator': 'incumbent',
      'CANDIDATE NAME (First)': 'first_name',
      'CANDIDATE NAME (Last)': 'last_name',
      'PARTY': 'party',
      'GENERAL VOTES ': 'votes',
      'GENERAL %': 'vote_share'
  })

  filtered_df = df.dropna(subset=['fec_id'])
  filtered_df = filtered_df[filtered_df['vote_share'] >= 0.01]
  filtered_df = filtered_df[filtered_df['district'] != 'S-UNEXPIRED TERM']
  filtered_df = filtered_df.drop('district', axis=1)

  filtered_df['year'] = year
  year_column = filtered_df.pop('year')
  filtered_df.insert(0, 'year', year_column)

  # Create a new column called 'Total Votes' which is the product of 'General Votes' and 'General %'
  filtered_df['total_votes'] = filtered_df['votes'] / filtered_df['vote_share']

  filtered_df[['votes', 'vote_share', 'total_votes']] = filtered_df[['votes', 'vote_share', 'total_votes']].round(3)

  def map_party(party):
      if party in ['D', 'R']:
          return party
      elif party == 'LIB':
          return 'L'
      else:
          return 'O'

  filtered_df['party'] = filtered_df['party'].apply(map_party)

  # Replace NA values in '(I)' column with 0
  filtered_df['incumbent'] = filtered_df['incumbent'].fillna(0)

  # Convert non-zero values to 1 in '(I)' column
  filtered_df.loc[filtered_df['incumbent'] != 0, 'incumbent'] = 1

  return filtered_df


In [5]:
os.chdir('/raw')
senate_2016 = process_senate_results(2016)
senate_2018 = process_senate_results(2018)
senate_2020 = process_senate_results(2020)

In [6]:
os.chdir('/content')
senate_results = pd.concat([senate_2016, senate_2018, senate_2020], ignore_index=True)
senate_results

,year,state,fec_id,incumbent,first_name,last_name,party,votes,vote_share,total_votes
0,2016,AL,S6AL00013,1,Richard C.,Shelby,R,1335104.0,0.640,2087444.0
1,2016,AL,S6AL00302,0,Ron,Crumpton,D,748709.0,0.359,2087444.0
2,2016,AK,S4AK00099,1,Lisa,Murkowski,R,138149.0,0.444,311441.0
3,2016,AK,S0AK00121,0,Joe,Miller,L,90825.0,0.292,311441.0
4,2016,AK,S8AK00082,0,Ray,Metcalfe,D,36200.0,0.116,311441.0
...,...,...,...,...,...,...,...,...,...,...
320,2020,WV,S4WV00159,1,Shelley Moore,Capito,R,547454,0.703,778918.0
321,2020,WV,S0WV00215,1,Paula Jean,Swearengin,D,210309,0.270,778918.0
322,2020,WV,S0WV00264,0,David,Moran,L,21155,0.027,778918.0
323,2020,WY,S0WY00137,0,Cynthia M.,Lummis,R,198100,0.726,272937.0


In [7]:
state_year_counts = senate_results.groupby('state')['year'].nunique()
states_with_more_than_2_years = state_year_counts[state_year_counts > 2].index.tolist()
states_with_more_than_2_years

['AZ']

In [11]:
senate_results_no_gender = senate_results[(senate_results['year'] != 2020) | (senate_results['state'] != 'AZ')]
senate_results_no_gender

,year,state,fec_id,incumbent,first_name,last_name,party,votes,vote_share,total_votes
0,2016,AL,S6AL00013,1,Richard C.,Shelby,R,1335104.0,0.640,2087444.0
1,2016,AL,S6AL00302,0,Ron,Crumpton,D,748709.0,0.359,2087444.0
2,2016,AK,S4AK00099,1,Lisa,Murkowski,R,138149.0,0.444,311441.0
3,2016,AK,S0AK00121,0,Joe,Miller,L,90825.0,0.292,311441.0
4,2016,AK,S8AK00082,0,Ray,Metcalfe,D,36200.0,0.116,311441.0
...,...,...,...,...,...,...,...,...,...,...
320,2020,WV,S4WV00159,1,Shelley Moore,Capito,R,547454,0.703,778918.0
321,2020,WV,S0WV00215,1,Paula Jean,Swearengin,D,210309,0.270,778918.0
322,2020,WV,S0WV00264,0,David,Moran,L,21155,0.027,778918.0
323,2020,WY,S0WY00137,0,Cynthia M.,Lummis,R,198100,0.726,272937.0


# Save to current directory

In [12]:
senate_results_no_gender.to_excel("senate_results_no_gender.xlsx", index=False)

# add gender and age manually then upload to github as "senate_results"

# Download text files with committee id information. This will be used to connect the information from the different
### https://www.fec.gov/data/browse-data/?tab=bulk-datadataframes.

In [ ]:
def ccl(year, source_url, download_directory, last_two_digits):

  filename = source_url.split('/')[-1]

  # Send a GET request to the URL
  response = requests.get(source_url)

  if response.status_code == 200:
      # Save the content of the response (the zip file) to a file
      with open(filename, 'wb') as f:
          f.write(response.content)

  else:
      pass

  zip_file_name = f'ccl{last_two_digits}.zip'
  extract_directory = '/raw'

  with zipfile.ZipFile(zip_file_name, 'r') as zip_ref:
    for file_info in zip_ref.infolist():

      # Extract all files in the zip file
      zip_ref.extractall(extract_directory)

      # Custom name for the extracted file (assuming it's a text file)
      custom_name = f"linkages_{year}.txt"  # Change this to whatever custom name you want

      # Rename the extracted file
      extracted_file_path = os.path.join(extract_directory, file_info.filename)
      new_file_path = os.path.join(extract_directory, custom_name)
      os.rename(extracted_file_path, new_file_path)

      # List the extracted files
      extracted_files = os.listdir('/raw')
      extracted_files

In [ ]:
years = list(range(2004,2048))[::2]
download_directory = "/raw"
for year in years:
  year = str(year)

  last_two_digits = year[-2:]

  source_url = f'https://www.fec.gov/files/bulk-downloads/{year}/ccl{last_two_digits}.zip'
  response = requests.get(source_url)
  if response.status_code == 200:
      ccl(year, source_url, download_directory, last_two_digits)
  else:
      print(f"They havn't released info for {year} yet!")

They havn't released info for 2026 yet!
They havn't released info for 2028 yet!
They havn't released info for 2030 yet!
They havn't released info for 2032 yet!
They havn't released info for 2034 yet!
They havn't released info for 2036 yet!
They havn't released info for 2038 yet!
They havn't released info for 2040 yet!
They havn't released info for 2042 yet!
They havn't released info for 2044 yet!
They havn't released info for 2046 yet!


In [ ]:
os.chdir('/raw')

column_names = ['fec_id', 'eh1', 'year', 'committee_id', 'eh2', 'eh3', 'eh4']

# Read the data from each file into a DataFrame
df1 = pd.read_csv('linkages_2016.txt', sep='|', names=column_names)
df2 = pd.read_csv('linkages_2018.txt', sep='|', names=column_names)
df3 = pd.read_csv('linkages_2020.txt', sep='|', names=column_names)

# Concatenate the DataFrames verticallyv
combined_df = pd.concat([df1, df2, df3], ignore_index=True)
combined_df.drop(['eh1', 'eh2', 'eh3', 'eh4'], axis=1, inplace=True)
df2 = combined_df[combined_df['fec_id'].str.startswith('S')]
# Display the combined DataFrame
print(df2)


          fec_id  year committee_id
5159   S0AK00121  2016    C00522730
5160   S0AK00154  2016    C00489377
5161   S0AR00028  2016    C00543603
5162   S0AR00028  2016    C00366401
5163   S0AR00028  2016    C00512053
...          ...   ...          ...
20640  S8WV00143  2020    C00681163
20641  S8WV00143  2020    C00651075
20642  S8WV00143  2020    C00689471
20643  S8WY00189  2020    C00663146
20644  S8WY00197  2020    C00671776

[3636 rows x 3 columns]


# keep only the rows that we have in the first dataframe

In [ ]:
unique_fec_ids = senate_results['fec_id'].unique().tolist()
filtered_df2 = df2[df2['fec_id'].isin(unique_fec_ids)]
filtered_df2

,fec_id,year,committee_id
5159,S0AK00121,2016,C00522730
5170,S0AR00150,2016,C00576280
5171,S0AR00150,2016,C00614529
5172,S0AR00150,2016,C00476317
5179,S0CA00199,2016,C00539890
...,...,...,...
20639,S8WV00143,2020,C00674770
20640,S8WV00143,2020,C00681163
20641,S8WV00143,2020,C00651075
20642,S8WV00143,2020,C00689471


# Download to excel

In [ ]:
os.chdir('/content')
filtered_df2.to_excel("id_linkages.xlsx", index=False)